In [6]:
import pandas as pd
import os
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
# start_time = time.time()
# df = pd.read_hdf('five_min_frame.hdf','five_min_frame')
# print "total time: %s" % (time.time() - start_time)

total time: 105.896847963


In [9]:
# filter to main line stations only
# df_ml = df[df['Lane Type'] == 'ML']

In [10]:
# df_ml.to_hdf('2015_ml_only.hdf', 'ml_only', mode='w')

In [48]:
# df_core = df_ml[['Timestamp', 'Station', 'Freeway #', 'Direction', '% Observed', 'Total Flow', 'Avg Occupancy',
#                  'Avg Speed']]
# df_core.to_hdf('2015_ml_core.hdf', 'ml_core', mode='w')

In [12]:
df_core = pd.read_hdf('2015_ml_core.hdf', 'ml_core')

82226723

In [13]:
no_dup_keep_last = pd.read_hdf('meta_2015.hdf', 'meta_2015')

In [39]:
to_loop = no_dup_keep_last.groupby(['Fwy', 'Dir'])['ID'].count().reset_index()[['Fwy', 'Dir']].values

In [116]:
daymap = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

def add_date_time_weekday( df ):
    """
    Josh's add date_time function
    """
    df.loc[:,'Timestamp2'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')
    df.loc[:,'Time'] = df['Timestamp2'].apply(lambda x:x.time())
    df.loc[:,'Date'] = df['Timestamp2'].apply(lambda x:x.date())
    df.loc[:,'Weekday'] = df['Date'].apply( lambda x: daymap[x.weekday()] )
    return df

In [128]:
def create_freeway_day_pivot(Fwy, Dir, stations_with_date=None):
    """
    This function is designed to pivot a particular freeway / direction
    """
    print "pivoting Fwy: %s Dir: %s" % (Fwy, Dir)
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    weekends = ['Saturday', 'Sunday']
    
    if Dir == "N":
        sort_order = ('Abs_PM', True)        
    elif Dir == "S":
        sort_order = ('Abs_PM', False)        
    elif Dir == "E":
        sort_order = ('Abs_PM', True)        
    elif Dir == "W":
        sort_order = ('Abs_PM', False) 

    if not stations_with_date:
        ret = no_dup_keep_last[(no_dup_keep_last.Fwy == Fwy) & (no_dup_keep_last.Dir == Dir)]\
            .sort_values(by=sort_order[0], ascending=sort_order[1])
        ret.index = np.arange(0, ret.shape[0])
        station_list = list(ret.ID.values)

        stations = df_core.Station.isin(station_list)
        df_stations = df_core[stations]

        # add date information to the dataframe of only the stations for this particular freeway / direction
        stations_with_date = add_date_time_weekday( df_stations )
    
    for day in daymap.values():
        print "day: %s" % day
        df = stations_with_date[stations_with_date.Weekday == day]

        # note: only creating flow at this point...
        pivoted = df.pivot_table(index='Station', columns='Time', values='Total Flow', aggfunc=np.mean)
        name = 'Flow_2015_%s_%s_%s' % (Fwy, Dir, day)
        pivoted.to_hdf('%s.hdf' % (name), name, mode='w')
    
    for partition, part_name in [(weekdays, 'Weekdays'), (weekends, 'Weekends')]:
        print "part_name: %s" % part_name
        partition_df = stations_with_date[ stations_with_date.Weekday.isin(partition) ]
        pivoted = partition_df.pivot_table(index='Station', columns='Time', values='Total Flow',
                                           aggfunc=np.mean)
        name = 'Flow_2015_%s_%s_%s' % (Fwy, Dir, part_name)
        pivoted.to_hdf('%s.hdf' % (name), name, mode='w')

    return stations_with_date

In [ ]:
stations_with_date = None
for Fwy, Dir in to_loop:
    start_time = time.time()
    stations_with_date = create_freeway_day_pivot(Fwy, Dir, stations_with_date)
    print "time: %s" % (time.time() - start_time)

pivoting Fwy: 5 Dir: N


In [120]:
pivoted.head()

Timestamp,00:00:00,00:05:00,00:10:00,00:15:00,00:20:00,00:25:00,00:30:00,00:35:00,00:40:00,00:45:00,...,23:10:00,23:15:00,23:20:00,23:25:00,23:30:00,23:35:00,23:40:00,23:45:00,23:50:00,23:55:00
Station,,,,,,,,,,,,,,,,,,,,,
1108437,103.435616,102.284932,96.769863,95.427397,90.989041,87.153425,83.709589,80.956164,77.468493,74.986301,...,154.209945,147.795580,144.878453,140.397790,134.359116,130.245856,125.187845,122.129834,116.179558,110.538674
1108507,108.347945,106.345205,101.731507,98.917808,96.054795,91.539726,87.273973,84.041096,82.010959,78.016438,...,162.732044,158.317680,153.911602,148.983425,142.524862,137.765193,132.395028,128.511050,122.220994,116.823204
1108509,107.863014,102.073973,101.246575,96.912329,94.282192,90.320548,85.739726,82.520548,79.583562,76.375342,...,160.961326,156.790055,151.502762,146.096685,141.160221,134.765193,131.212707,125.582873,120.790055,114.265193
1108512,115.098630,108.641096,106.424658,102.024658,100.509589,95.534247,91.158904,87.624658,83.783562,81.652055,...,173.116022,168.395028,161.958564,157.019337,150.160221,145.033149,140.425414,133.392265,128.861878,120.988950
1108514,118.715068,111.372603,110.210959,104.230137,101.345205,97.791781,92.723288,88.887671,85.945205,82.172603,...,179.593923,174.546961,167.234807,162.475138,155.696133,148.964088,144.226519,137.303867,132.585635,124.367403


In [ ]:
2015_5_N_Monday_Flow.hdf

In [131]:
fiveN_Monday = pd.read_hdf('Flow_2015_5_N_Weekends.hdf', 'Flow_2015_5_N_Weekends')

In [132]:
fiveN_Monday

Time,00:00:00,00:05:00,00:10:00,00:15:00,00:20:00,00:25:00,00:30:00,00:35:00,00:40:00,00:45:00,...,23:10:00,23:15:00,23:20:00,23:25:00,23:30:00,23:35:00,23:40:00,23:45:00,23:50:00,23:55:00
Station,,,,,,,,,,,,,,,,,,,,,
1108437,142.461538,141.057692,133.442308,132.115385,123.576923,119.846154,115.240385,112.115385,104.259615,101.673077,...,192.990291,182.747573,179.252427,177.165049,169.757282,166.320388,158.796117,153.941748,145.912621,142.000000
1108507,151.221154,147.721154,140.576923,137.134615,130.000000,124.394231,120.480769,115.365385,111.528846,104.336538,...,202.155340,194.708738,189.368932,186.446602,178.213592,174.572816,166.766990,161.233010,153.854369,149.990291
1108509,149.740385,140.346154,141.865385,132.548077,128.865385,122.278846,116.798077,113.038462,109.490385,102.413462,...,200.533981,191.466019,185.679612,181.436893,177.737864,168.388350,165.475728,156.737864,152.019417,144.514563
1108512,162.317308,153.836538,151.451923,144.557692,141.673077,132.182692,127.153846,123.403846,118.586538,112.567308,...,214.097087,208.834951,199.563107,194.951456,189.883495,182.067961,178.873786,168.980583,163.203883,154.174757
1108514,170.990385,160.221154,159.375000,149.211538,146.375000,138.451923,131.730769,127.307692,124.288462,114.692308,...,227.349515,221.300971,210.766990,207.776699,201.000000,191.815534,187.000000,178.563107,170.252427,163.029126
1108547,162.413462,156.980769,151.028846,147.778846,144.192308,136.942308,130.855769,123.096154,118.413462,114.615385,...,204.194175,202.427184,194.592233,187.184466,175.708738,171.922330,161.941748,156.796117,148.126214,143.029126
1108603,154.048077,147.413462,143.211538,141.778846,138.134615,129.240385,123.500000,121.355769,121.413462,114.846154,...,172.233010,167.796117,160.922330,158.650485,151.601942,145.000000,148.407767,141.456311,135.514563,131.155340
1108611,135.019231,128.567308,124.230769,124.567308,118.980769,110.413462,107.673077,106.394231,102.307692,98.413462,...,155.631068,149.174757,143.844660,139.951456,136.038835,131.514563,128.533981,122.854369,118.029126,114.058252
1108613,157.951923,151.778846,149.182692,146.509615,139.509615,133.538462,127.903846,123.567308,118.509615,114.461538,...,192.417476,183.203883,177.203883,170.572816,164.097087,157.495146,153.495146,148.747573,142.330097,133.349515
